In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from langdetect import detect
from googletrans import Translator
import os
import pickle

if not os.path.exists('data'):
    os.mkdir('data')

%matplotlib

Using matplotlib backend: Qt5Agg


סוג אבחנה diagnosis_type
מצב תפקודי בקבלה status_at_reception
מצב תפקויד לפני אשפוז status_before_hospitalization
הגיע למחלקה מ: arrived_from
גורם מפנה reffered_by
מקום לקיחת דגימה sample_taken_from
בדיקה-אוטולאב autolab
תאריך בדיקה test_data
מספר מדבקה sticker_number
תאריך ושעת אישור בדיקה test_auth_time
גיל age_
גיל נוכחי current_age
מין gender_
תאריך לידה dob
חיידק bacteria
תוצאת חיידק -דיווח מקורי bacteria_original_result
אנטיביוטיקה antibiotic
תוצאת אנטיביוטיקה-דיווח מקורי antibiotic_original_result
רגישות לאנטיביוטיקה antibiotic_sensitivity


In [3]:
trans = Translator()
def parser(s):
#     try:
#         det = trans.detect(s)
#     except Exception as e:
#         print('Fail for', s, type(s), str(e))
#         return s
#     if det.confidence < 0.95 or det.lang !='iw':
#         return s
    return trans.translate(s).text

cols_translate = [
        'Gender',
        'Name',
        'diagnosis_type',	
        'status_at_reception',
        'status_before',
        'arrived_from',
        'sample_taken', 
        'autolab',
    ]    
    
def parse_dic(df):
    trans=Translator()
    filename = '/home/yairi/antres/dictionary.pickle' ## CHANGE THIS
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            dic = pickle.load(f)
    else:
        dic = dict()
    words = set()
    for col in cols_translate:
        if not col in df.columns:
            continue
        words = words.union(df[col].unique())

    words = list(words.difference(dic.keys()))
    words = [word for word in words if not pd.isnull(word)]
    print('Words we ask Google to translate', words)
    new_dict = dict(zip(words, [tr.text for tr in trans.translate(words)]))
    dic.update(new_dict)
    with open(filename, 'wb') as f:
        pickle.dump(dic, f)
    return dic

#parse_dic(admission_data)
#parse_dic(cultures)

In [4]:
cultures.bacteria_original_result.unique()

array(['.', 'CRE', 'ESBL POSITIVE', 'GENTA.-HLR POS.', 'ESBL POSTIVE',
       'ראה הערה', 'pseud. sp', 'mucoid', 'Over growth', 'St.coag.neg',
       'St.coag neg', 'סוג 2', 'חשוד ל-CRE', 'Lactococcus', 'Strep?',
       'ESBL+', 'KPC+', 'שני סוגים', 'E. coli ESBL+', 'Staph. coag(-)',
       'MRSA+', 'Moderate growth', 'Mucoid', 'Ps.aeruginosa', 'צמיחה רבה',
       'וורוד', 'סוג 1', 'Ps. aeruginosa', 'P.putida', nan,
       'St. epidermidis', 'Staph', 'Ps. luteola', 'Staph coag(-)',
       'Pantoea sp.', 'מספר 1', 'מספר 2', 'מוקואיד', 'סוג שני',
       'Strep. parasanguinis', 'P.oryzihabitans', 'St.coag(-)זן2',
       'aeruginosa', 'xylosoxidans', 'staph coag (-)', 'St. coag(-)זן 2',
       'מושבה אחת', 'פנוטיפ רירי', 'Aeruginosa', '2 סוגים',
       'Staph. warneri', 'Staph. hominis', 'Untypeable', 'CRE+', '/',
       'staph hominis', 'C. koseri', 'Raoultella', 'זן 2', 'זן 1',
       'CRE(+)', 'St. Pasteuri', 'CRE (+)', 'Enterococci זן2', 'מס 1',
       'מס 2', 'Sal. Enteritidis', 'Stre

In [12]:
admission_data.columns

Index(['RANDID', 'Gender', 'age', 'ER_Admission_No', 'ER_Arrival_Date',
       'Release_Date', 'Name', 'Diagnosis', 'Diagnosis_Date', 'Name.1', 'Icd',
       'diagnosis_type', 'status_at_reception',
       'status_before_hospitalization', 'arrived_from', 'grpd_diagnosis'],
      dtype='object')

In [14]:
cultures.head()

,RANDID,reffered_by,sample_taken_from,autolab,test_data,sticker_number,test_auth_time,age_,current_age,gender_,dob,bacteria,bacteria_original_result,antibiotic,antibiotic_original_result,antibiotic_sensitivity
0,XC101310,השתלות ריאה,כיח,Sputum culture result,01/01/2009,7000025,04/01/2009 11:05,67,74,זכר,28/08/1941,Escherichia coli,.,Amikacin,S,Susceptible
1,XC101310,השתלות ריאה,כיח,Sputum culture result,01/01/2009,7000025,04/01/2009 11:05,67,74,זכר,28/08/1941,Escherichia coli,.,Amoxicillin/Clavul A,S,Susceptible
2,XC101310,השתלות ריאה,כיח,Sputum culture result,01/01/2009,7000025,04/01/2009 11:05,67,74,זכר,28/08/1941,Escherichia coli,.,Ampicillin,R,Resistant
3,XC101310,השתלות ריאה,כיח,Sputum culture result,01/01/2009,7000025,04/01/2009 11:05,67,74,זכר,28/08/1941,Escherichia coli,.,Ceftazidime,S,Susceptible
4,XC101310,השתלות ריאה,כיח,Sputum culture result,01/01/2009,7000025,04/01/2009 11:05,67,74,זכר,28/08/1941,Escherichia coli,.,Cefuroxime,S,Susceptible


In [19]:
cultures.loc[np.where((cultures.antibiotic_original_result == 'S') != (cultures.antibiotic_sensitivity == 'Susceptible'))[0]]

,RANDID,reffered_by,sample_taken_from,autolab,test_data,sticker_number,test_auth_time,age_,current_age,gender_,dob,bacteria,bacteria_original_result,antibiotic,antibiotic_original_result,antibiotic_sensitivity
1339,XC101965,פנימית ב בילינ,משטח רקטלי,NaN,17/02/2009,5200405,22/02/2009 02:30,52,59,נקבה,08/01/1957,Klebsiella pneumoniae,ESBL POSITIVE,Ertapenem,2,Susceptible
1380,XC101249,נוירוכירורגיה בילינ,פצע,Wound culture result,17/02/2009,9001695,19/02/2009 12:12,75,82,זכר,01/12/1933,Providencia rettgeri,.,Sulfamethoxa/Trimeth,0.25,Susceptible
2254,XC102552,פנימית ד בילינ,NaN,Aerobic boold result,20/03/2009,4515160,23/03/2009 08:24,85,89,זכר,22/11/1923,Streptococcus bovis II,ראה הערה,Ampicillin,0.064,Susceptible
2258,XC102552,פנימית ד בילינ,NaN,Aerobic boold result,20/03/2009,4515160,23/03/2009 08:24,85,89,זכר,22/11/1923,Streptococcus bovis II,ראה הערה,Penicillin,0.064,Susceptible
2550,XC101038,פנימית ד בילינ,NaN,Anaerobic blood result,01/04/2009,4517153,03/04/2009 08:52,71,74,נקבה,05/05/1937,Streptococcus group B,.,Ampicillin,0.094,Susceptible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276000,XC100951,מיון פנימי בילינ,שתן,First isolate,23/11/2014,4025224,25/11/2014 09:48,44,46,זכר,24/04/1970,Klebsiella pneumoniae,.,Fosfomycin,NaN,Susceptible
276002,XC100951,מיון פנימי בילינ,שתן,First isolate,23/11/2014,4025224,25/11/2014 09:48,44,46,זכר,24/04/1970,Klebsiella pneumoniae,.,Meropenem,NaN,Susceptible
276003,XC100951,מיון פנימי בילינ,שתן,First isolate,23/11/2014,4025224,25/11/2014 09:48,44,46,זכר,24/04/1970,Klebsiella pneumoniae,.,Nitrofurantoin,NaN,Susceptible
276004,XC100951,מיון פנימי בילינ,שתן,First isolate,23/11/2014,4025224,25/11/2014 09:48,44,46,זכר,24/04/1970,Klebsiella pneumoniae,.,Ofloxacin,NaN,Susceptible


In [29]:
admission.head()


NameError: name 'admission' is not defined

In [6]:
df = pd.DataFrame( data = [ [1,2,3], [4,4,2]])

In [7]:
df.replace({1:'OK'})

,0,1,2
0,OK,2,3
1,4,4,2


In [8]:
df

,0,1,2
0,1,2,3
1,4,4,2
